# BIGMART SALES PREDICTION(REGRESSION MODEL)

In [1]:
!pip install cufflinks --upgrade

Requirement already up-to-date: cufflinks in c:\users\lipsita_senapati\appdata\local\programs\python\python36\lib\site-packages (0.17.0)


In [26]:
# importing required libraries
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_squared_error
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import matplotlib.pyplot as plt
import seaborn as sns
from plotly.offline import init_notebook_mode, download_plotlyjs, iplot
import cufflinks as cf
init_notebook_mode(connected=True)
cf.go_offline()
import warnings
warnings.filterwarnings('ignore')

In [27]:
#Importing the datasets
train_data = pd.read_csv(r'C:\Users\LIPSITA_SENAPATI\Desktop\Bigmart\train.csv')
test_data = pd.read_csv(r'C:\Users\LIPSITA_SENAPATI\Desktop\Bigmart\test.csv')




In [28]:
train_data.head()


,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type,Item_Outlet_Sales
0,FDA15,9.30,Low Fat,0.016047,Dairy,249.8092,OUT049,1999,Medium,Tier 1,Supermarket Type1,3735.1380
1,DRC01,5.92,Regular,0.019278,Soft Drinks,48.2692,OUT018,2009,Medium,Tier 3,Supermarket Type2,443.4228
2,FDN15,17.50,Low Fat,0.016760,Meat,141.6180,OUT049,1999,Medium,Tier 1,Supermarket Type1,2097.2700
3,FDX07,19.20,Regular,0.000000,Fruits and Vegetables,182.0950,OUT010,1998,NaN,Tier 3,Grocery Store,732.3800
4,NCD19,8.93,Low Fat,0.000000,Household,53.8614,OUT013,1987,High,Tier 3,Supermarket Type1,994.7052


In [29]:
test_data.head()

,Item_Identifier,Item_Weight,Item_Fat_Content,Item_Visibility,Item_Type,Item_MRP,Outlet_Identifier,Outlet_Establishment_Year,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,FDW58,20.750,Low Fat,0.007565,Snack Foods,107.8622,OUT049,1999,Medium,Tier 1,Supermarket Type1
1,FDW14,8.300,reg,0.038428,Dairy,87.3198,OUT017,2007,NaN,Tier 2,Supermarket Type1
2,NCN55,14.600,Low Fat,0.099575,Others,241.7538,OUT010,1998,NaN,Tier 3,Grocery Store
3,FDQ58,7.315,Low Fat,0.015388,Snack Foods,155.0340,OUT017,2007,NaN,Tier 2,Supermarket Type1
4,FDY38,NaN,Regular,0.118599,Dairy,234.2300,OUT027,1985,Medium,Tier 3,Supermarket Type3


In [30]:
train_data.head()
# shape of the dataset
print('\nShape of training data :',train_data.shape)
print('\nShape of testing data :',test_data.shape)



Shape of training data : (8523, 12)

Shape of testing data : (5681, 11)


In [31]:
#Removing Unwanted Columns/Features
try:
    train_data.drop(labels=['Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
    test_data.drop(labels=['Item_Identifier', 'Outlet_Identifier', 'Outlet_Establishment_Year'], axis=1, inplace=True)
except Exception as e:
    pass

In [32]:
train_data.head()
# shape of the dataset
print('\nShape of training data :',train_data.shape)
print('\nShape of testing data :',test_data.shape)





Shape of training data : (8523, 9)

Shape of testing data : (5681, 8)


In [33]:
#visualising categorical variables
categorial_features = train_data.select_dtypes(include=[np.object])
categorial_features.head(2)

,Item_Fat_Content,Item_Type,Outlet_Size,Outlet_Location_Type,Outlet_Type
0,Low Fat,Dairy,Medium,Tier 1,Supermarket Type1
1,Regular,Soft Drinks,Medium,Tier 3,Supermarket Type2


In [34]:
#visualising numerical variables
numerical_features = train_data.select_dtypes(include=[np.number])
numerical_features.head(2)

,Item_Weight,Item_Visibility,Item_MRP,Item_Outlet_Sales
0,9.30,0.016047,249.8092,3735.1380
1,5.92,0.019278,48.2692,443.4228


In [35]:
# Replacing the duplicates with its original categories
# Making Correction in 'Item_Fat_Content' column
train_data["Item_Fat_Content"] = train_data["Item_Fat_Content"].str.replace("reg","Regular")
train_data["Item_Fat_Content"] = train_data["Item_Fat_Content"].str.replace("LF","Low Fat")
train_data["Item_Fat_Content"] = train_data["Item_Fat_Content"].str.replace("low fat","Low Fat")
train_data["Item_Fat_Content"].unique()

array(['Low Fat', 'Regular'], dtype=object)

In [36]:
#visualising categorical variables
temp_df = train_data.isnull().sum().reset_index()
temp_df['Percentage'] = (temp_df[0]/len(train_data))*100
temp_df.columns = ['Column Name', 'Number of null values', 'Null values in percentage']
print(f"The length of dataset is \t {len(train_data)}")
temp_df

The length of dataset is 	 8523


,Column Name,Number of null values,Null values in percentage
0,Item_Weight,1463,17.165317
1,Item_Fat_Content,0,0.000000
2,Item_Visibility,0,0.000000
3,Item_Type,0,0.000000
4,Item_MRP,0,0.000000
5,Outlet_Size,2410,28.276428
6,Outlet_Location_Type,0,0.000000
7,Outlet_Type,0,0.000000
8,Item_Outlet_Sales,0,0.000000


In [37]:
!pip install missingpy

In [38]:
# Imputing the nan values using Knn Imputer for Item_Weight Column
from missingpy import KNNImputer
kn= KNNImputer(weights='distance')
new_weight= kn.fit_transform(train_data["Item_Weight"].values.reshape(-1,1))
train_data["Item_Weight"]=new_weight

In [39]:
# Substituting the missing values with mean for Item_Weight Column
train_data["Item_Weight"]=train_data["Item_Weight"].fillna(np.mean(train_data["Item_Weight"]))

In [40]:
#Imputing nan values using fillna "backwordFill" method and analysing the nature of Item_Outlet_Sales after Imputation.
train_data["Outlet_Size"]=train_data["Outlet_Size"].fillna(method="bfill")
train_data.pivot_table(index="Outlet_Size",values="Item_Outlet_Sales")

,Item_Outlet_Sales
Outlet_Size,
High,2147.162776
Medium,2441.709463
Small,1882.829379


In [41]:
# Counting the values
count = train_data['Outlet_Size'].value_counts().reset_index()
count.iplot(kind='bar', color='deepskyblue', x='index', y='Outlet_Size', title='High VS Mediun VS Small',
           xTitle='Size', yTitle='Frequency')

In [42]:
train_data['Outlet_Size'].fillna(value='Medium', inplace= True)
test_data['Outlet_Size'].fillna(value='Medium', inplace= True)

In [43]:
# Let us Import the Important Libraries  to train our Model for Machine Learning 
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor

from sklearn.preprocessing import LabelEncoder, OneHotEncoder # To deal with Categorical Data in Target Vector.
from sklearn.model_selection import train_test_split  # To Split the dataset into training data and testing data.
from sklearn.model_selection import cross_val_score   # To check the accuracy of the model.
from sklearn.preprocessing import Imputer   # To deal with the missing values
from sklearn.preprocessing import StandardScaler   # To appy scaling on the dataset.

In [44]:
# Let us create feature matrix and Target Vector.
x_train = train_data.iloc[:, :-1].values    # Features Matrix
y_train = train_data.iloc[:,-1].values   # Target Vector
x_test = test_data.values    # Features Matrix

In [45]:
#Dealing with Missing data
imputer = Imputer()
x_train[:,[0]] = imputer.fit_transform(x_train[:,[0]])
x_test[:,[0]] = imputer.fit_transform(x_test[:,[0]])

In [46]:
#Dealing With the Categorical Values in Features/Columns
labelencoder_x = LabelEncoder()
x_train[:, 1 ] = labelencoder_x.fit_transform(x_train[:,1 ])
x_train[:, 3 ] = labelencoder_x.fit_transform(x_train[:,3 ])
x_train[:, 5 ] = labelencoder_x.fit_transform(x_train[:,5 ])
x_train[:, 6 ] = labelencoder_x.fit_transform(x_train[:,6 ])
x_train[:, 7 ] = labelencoder_x.fit_transform(x_train[:,7 ])


y = labelencoder_x.fit_transform(y_train)

#this is need to done when we have more than two categorical values.
onehotencoder_x = OneHotEncoder(categorical_features=[3,5,6,7]) 
x_train = onehotencoder_x.fit_transform(x_train).toarray()

# Let's apply same concept on test set.
x_test[:, 1 ] = labelencoder_x.fit_transform(x_test[:,1 ])
x_test[:, 3 ] = labelencoder_x.fit_transform(x_test[:,3 ])
x_test[:, 5 ] = labelencoder_x.fit_transform(x_test[:,5 ])
x_test[:, 6 ] = labelencoder_x.fit_transform(x_test[:,6 ])
x_test[:, 7 ] = labelencoder_x.fit_transform(x_test[:,7 ])


#this is need to done when we have more than two categorical values.
onehotencoder_x = OneHotEncoder(categorical_features=[3,5,6,7]) 
x_test = onehotencoder_x.fit_transform(x_test).toarray()



In [47]:
#Now time to Apply Feature Scaling on Feature matrix
sc_X=StandardScaler()
x_train=sc_X.fit_transform(x_train)
x_test = sc_X.fit_transform(x_test)

#### DIMENSIONALITY REDUCTION
We are doing this to reduce the number of dimensions/features in the dataset.
The features which have less effect on the prediction , we will remove those features.
It also boosts the process.
It saves time.
Here we will use Principal Component Analysis (PCA) with 'rbf' kernel.

In [48]:
from sklearn.decomposition import PCA
pca = PCA(n_components=None)
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)
explained_variance = pca.explained_variance_ratio_
explained_variance

array([1.07435395e-01, 7.06779829e-02, 5.70582693e-02, 4.82279965e-02,
       3.97253836e-02, 3.89516984e-02, 3.87912142e-02, 3.77754090e-02,
       3.69033106e-02, 3.65911798e-02, 3.64299235e-02, 3.61849487e-02,
       3.59178407e-02, 3.51106774e-02, 3.47336718e-02, 3.45149737e-02,
       3.43034392e-02, 3.41749035e-02, 3.40243157e-02, 3.33646755e-02,
       3.13196424e-02, 2.98467645e-02, 2.95990947e-02, 2.37944495e-02,
       2.21354752e-02, 2.40736441e-03, 3.53459725e-30, 2.78267094e-31,
       2.09833592e-31, 7.87225799e-32])

In [49]:
pca = PCA(n_components=25)
x_train = pca.fit_transform(x_train)
x_test = pca.fit_transform(x_test)

In [50]:
# Multi-linear regression Model.
regressor_multi = LinearRegression()
regressor_multi.fit(x_train,y)

# Let us check the accuray
accuracy = cross_val_score(estimator=regressor_multi, X=x_train, y=y,cv=10)
print(f"The accuracy of the Multi-linear Regressor Model is \t {accuracy.mean()}")
print(f"The deviation in the accuracy is \t {accuracy.std()}")

The accuracy of the Multi-linear Regressor Model is 	 0.6276344343807515
The deviation in the accuracy is 	 0.024311359056437085


In [51]:
# Fitting polynomial regression to dataset
from sklearn.preprocessing import PolynomialFeatures
poly_reg=PolynomialFeatures(degree=4) #These 3 steps are to convert X matrix into X polynomial
x_poly=poly_reg.fit_transform(x_train) #matrix. 
regressor_poly=LinearRegression()
regressor_poly.fit(x_poly,y)

# Let us check the accuray
accuracy = cross_val_score(estimator=regressor_poly, X=x_train, y=y,cv=10)
print(f"The accuracy of the Polynomial Regression Model is \t {accuracy.mean()}")
print(f"The deviation in the accuracy is \t {accuracy.std()}")

The accuracy of the Polynomial Regression Model is 	 0.6276344343807515
The deviation in the accuracy is 	 0.024311359056437085


In [ ]:
import xgboost as xgb

from xgboost import XGBClassifier
from sklearn.model_selection import GridSearchCV
model = XGBClassifier()
model.fit(x_train, y)

accuracy = cross_val_score(estimator=model, X=x_train, y=y, cv=5)
print(f"The accuracy of the xgboost Model is \t {accuracy.mean()}") 
print(f"The deviation in the accuracy is \t {accuracy.std()}")

In [ ]:
#Import Gaussian Naive Bayes model
from sklearn.naive_bayes import GaussianNB

#Create a Gaussian Classifier
model1 = GaussianNB()

# Train the model using the training sets
model1.fit(x_train, y)
accuracy = cross_val_score(estimator=model1, X=x_train, y=y, cv=10)
print(f"The accuracy of the Gaussian Naive Bayes Model is \t {accuracy.mean()}") 
print(f"The deviation in the accuracy is \t {accuracy.std()}")


In [ ]:
from sklearn.linear_model import LogisticRegression
mul_lr = LogisticRegression(multi_class='multinomial', solver='newton-cg').fit(x_train,y)
mul_lr.fit(x_train,y)
accuracy = cross_val_score(estimator=mul_lr, X=x_train, y=y, cv=5)
print(f"The accuracy of the Multinomial LogisticRegression Model is \t {accuracy.mean()}") 
print(f"The deviation in the accuracy is \t {accuracy.std()}")


In [52]:
y_pred = regressor_multi.predict(x_test)

y_pred[:5]

array([2533.10563608, 1828.22465952, -319.02723557, 2401.57922922,
       1407.85433935])